In [1]:
from functools import partial

from sepal_ui import widgetFactory as wf
from sepal_ui import widgetBinding as wb
from sepal_ui.scripts import utils

from scripts import run
from utils import messages as ms

In [2]:
#use a class to define your input and output in order to have mutable variables
class Gfc_export_IO:
    def __init__(self):
        
        #set up output
        self.clip_map = None
        
gfc_export_io = Gfc_export_IO()

In [3]:
#create the tile with the links
id_ = "gfc_export_widget"
title = "Results"
    
gfc_results = wf.Tile(id_, title, inputs=[''])

In [4]:
import ipyvuetify as v

#create the output alert 
gfc_export_output = wf.OutputWidget(ms.GFC_EXPORT)

#create the button that start your process
gfc_export_btn = wf.ProcessBtn('Export data')

#create a process tile
id_ = "gfc_export_widget"
title = 'Export the data'

gfc_export = wf.Tile(
    id_, 
    title, 
    btn=gfc_export_btn,  
    output=gfc_export_output,
    inputs=['']
)

#bind the button to the process by writing a custom function
def process_start(widget, event, data, output, gfc_results):
    
    #debug
    #asset = 'users/bornToBeAlive/aoi_PU'
    #threshold = 42
    #visualization = True
    
    asset = gfc_aoi_IO.assetId
    threshold = gfc_viz_io.threshold
    visualization = gfc_viz_io.visualization
    
    #toggle the loading button
    utils.toggleLoading(widget)
    
    #check inputs
    if not wb.checkInput(asset, output, ms.NO_AOI): return utils.toggleLoading(widget)
    if not wb.checkInput(threshold, output, ms.NO_THRESHOLD): return utils.toggleLoading(widget)
    if not wb.checkInput(visualization, output, ms.NO_VIZ): return utils.toggleLoading(widget)
    
    #retreive the data
    #launch any process you want
    clip_map, csv_file, result_layout = run.gfcExport(asset, threshold, output)
    
    #save the clip_map variable
    setattr(gfc_export_io, 'clip_map', clip_map)
    
    gfc_results.children[0].children = [gfc_results.children[0].children[0]] + result_layout
    
    #toggle the loading button
    return utils.toggleLoading(widget)

gfc_export_btn.on_event('click', partial(
    process_start, 
    output = gfc_export_output,
    gfc_results = gfc_results
))

In [5]:
#this tiles will only be displayed if you launch voila from this file 
gfc_export

Layout(align_center=True, children=[Card(children=[Html(children=['Export the data'], tag='h2'), Flex(children…

gdalwarp /home/prambaud/gfc_wrapper_results/gfc/PU_42_merged_gfc_map.tif /home/prambaud/gfc_wrapper_results/tmp/PU_42_merged_gfc_map_proj.tif -t_srs ESRI:54009
Usage: gdalwarp [--help-general] [--formats]

    [-s_srs srs_def] [-t_srs srs_def] [-to "NAME=VALUE"]* [-novshiftgrid]

    [-order n | -tps | -rpc | -geoloc] [-et err_threshold]

    [-refine_gcps tolerance [minimum_gcps]]

    [-te xmin ymin xmax ymax] [-tr xres yres] [-tap] [-ts width height]

    [-ovr level|AUTO|AUTO-n|NONE] [-wo "NAME=VALUE"] [-ot Byte/Int16/...] [-wt Byte/Int16]

    [-srcnodata "value [value...]"] [-dstnodata "value [value...]"] -dstalpha

    [-r resampling_method] [-wm memory_in_mb] [-multi] [-q]

    [-cutline datasource] [-cl layer] [-cwhere expression]

    [-csql statement] [-cblend dist_in_pixels] [-crop_to_cutline]

    [-of format] [-co "NAME=VALUE"]* [-overwrite]

    [-nomd] [-cvmd meta_conflict_value] [-setci] [-oo NAME=VALUE]*

    [-doo NAME=VALUE]*

    srcfile* dstfile



Available resam

RasterioIOError: /home/prambaud/gfc_wrapper_results/tmp/PU_42_merged_gfc_map_proj.tif: No such file or directory

In [6]:
#this tiles will only be displayed if you launch voila from this file 
gfc_results

Layout(align_center=True, children=[Card(children=[Html(children=['Results'], tag='h2'), Flex(children=[Layout…

In [18]:
!ls
!gdalwarp /home/prambaud/gfc_wrapper_results/gfc/PU_42_merged_gfc_map.tif /home/prambaud/gfc_wrapper_results/tmp/PU_42_merged_gfc_map_proj.tif -t_srs ESRI:54009
    

about_ui.ipynb	       gfc_map_ui.ipynb       README.md  utils
aoi_selector_ui.ipynb  mspa		      scripts
gfc_export_ui.ipynb    mspa_process_ui.ipynb  UI.ipynb
ERROR 1: PROJ: proj_get_authorities_from_database: Cannot find proj.db
ERROR 1: Translating source or target SRS failed:
ESRI:54009
Usage: gdalwarp [--help-general] [--formats]
    [-s_srs srs_def] [-t_srs srs_def] [-to "NAME=VALUE"]* [-novshiftgrid]
    [-order n | -tps | -rpc | -geoloc] [-et err_threshold]
    [-refine_gcps tolerance [minimum_gcps]]
    [-te xmin ymin xmax ymax] [-tr xres yres] [-tap] [-ts width height]
    [-ovr level|AUTO|AUTO-n|NONE] [-wo "NAME=VALUE"] [-ot Byte/Int16/...] [-wt Byte/Int16]
    [-srcnodata "value [value...]"] [-dstnodata "value [value...]"] -dstalpha
    [-r resampling_method] [-wm memory_in_mb] [-multi] [-q]
    [-cutline datasource] [-cl layer] [-cwhere expression]
    [-csql statement] [-cblend dist_in_pixels] [-crop_to_cutline]
    [-of format] [-co "NAME=VALUE"]* [-overwrite]
    [-nomd

In [24]:
!printenv



LC_ALL=en_US.UTF-8
GDAL_DATA=/usr/share/gdal
_=/usr/bin/printenv
LANG=en_US.UTF-8
SUDO_GID=0
HOSTNAME=0e8c22b71caa
SDKMAN_CANDIDATES_API=https://api.sdkman.io/2
KERNEL_LAUNCH_TIMEOUT=40
NODE_PATH=:/usr/lib/node_modules:/node_modules
USERNAME=prambaud
OPENSARKIT=/usr/local/lib/osk/opensarkit
JAVA_HOME=/usr/local/lib/sdkman/candidates/java/current
SUDO_COMMAND=/bin/bash -c python3 /usr/local/bin/jupyter-notebook --no-browser --allow-root --ip=0.0.0.0 --port=8888 --NotebookApp.token= --NotebookApp.base_url=/api/sandbox/jupyter/ --NotebookApp.notebook_dir=/home/prambaud
R_LIBS_USER=/home/prambaud/.R/library
S_COLORS=auto
GROOVY_HOME=/usr/local/lib/sdkman/candidates/groovy/current
PROJ_LIB=/usr/local/lib/python3.6/dist-packages/rasterio/proj_data
USER=prambaud
PAGER=cat
OST_DB=/usr/local/lib/osk/Database/OFST_db.sqlite
JPY_PARENT_PID=174
PWD=/home/prambaud/gfc_wrapper_python
HOME=/home/prambaud
CLICOLOR=1
SNAP_EXE=/usr/local/snap/bin/gpt.sepal
SUDO_USER=root
SEPAL=true
SDKMAN_DIR=/usr/local

In [23]:
import os

os.environ

environ{'LC_ALL': 'en_US.UTF-8',
        'GDAL_DATA': '/usr/share/gdal',
        'LANG': 'en_US.UTF-8',
        'SUDO_GID': '0',
        'HOSTNAME': '0e8c22b71caa',
        'SDKMAN_CANDIDATES_API': 'https://api.sdkman.io/2',
        'NODE_PATH': ':/usr/lib/node_modules:/node_modules',
        'USERNAME': 'prambaud',
        'OPENSARKIT': '/usr/local/lib/osk/opensarkit',
        'JAVA_HOME': '/usr/local/lib/sdkman/candidates/java/current',
        'SUDO_COMMAND': '/bin/bash -c python3 /usr/local/bin/jupyter-notebook --no-browser --allow-root --ip=0.0.0.0 --port=8888 --NotebookApp.token= --NotebookApp.base_url=/api/sandbox/jupyter/ --NotebookApp.notebook_dir=/home/prambaud',
        'R_LIBS_USER': '/home/prambaud/.R/library',
        'S_COLORS': 'auto',
        'GROOVY_HOME': '/usr/local/lib/sdkman/candidates/groovy/current',
        'USER': 'prambaud',
        'OST_DB': '/usr/local/lib/osk/Database/OFST_db.sqlite',
        'PWD': '/home/prambaud',
        'HOME': '/home/prambaud',
     

In [22]:
!unset PROJ_LIB